A

In [1]:
import sys
# input = lambda: sys.stdin.readline().rstrip()

pi = '314159265358979323846264338327'

for _ in range(int(input())):
    s = input()
    n = len(s)
    
    r = 0
    for i in range(n):
        if pi[i] == s[i]:
            r += 1
        else:
            break
            
    print(r)

3
1
0
31
2
3
1


B

In [4]:
import io, os, sys
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

for _ in range(int(input())):
    n, s, r = map(int, input().split())

    result = [0] * n
    result[-1] = s - r

    i = 0
    while r:
        r -= 1
        result[i] += 1

        i += 1
        if i == n - 1:
            i = 0

    print(*result)

4 9 5
2 2 1 4


C

In [9]:
import io, os, sys
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

for _ in range(int(input())):
    n = int(input())
    li = [list(map(int, input().split())) for _ in range(n)]

    st = set()
    temp = []
    for i in range(n):
        st.add(li[i][0])
        temp.append(li[i][0])

    fs_i = -1
    for v in st:
        if temp.count(v) == 1:
            fs_i = temp.index(v)
            st.discard(v)

            break

    print(*([st.pop()] + li[fs_i]))

4
4 2 1
4 2 3
2 1 3
4 1 3
4 2 1 3


D

In [11]:
import io, os, sys
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans

for _ in range(int(input())):
    n = int(input())
    li = list(map(int, input().split()))

    ms = SortedMultiset()

    for v in li:
        ms.add(v)

    c = 0
    while ms:
        c += 1
        cur = ms[0]
        while True:
            if cur in ms:
                ms.discard(cur)
                cur += 1
            else:
                break

    print(c)

5
4 2 2 3 4
3


E

In [29]:
import io, os, sys
input = io.BytesIO(os.read(0, os.fstat(0).st_size)).readline

for _ in range(int(input())):
    x = int(input())

    if x % 2 == 1:
        print(-1)
    else:
        a_xor_b = bin(x)[2:]
        a_and_b = '0' + a_xor_b[:-1]

        n = len(a_xor_b)

        a = 0
        b = 0
        cur = 1

        ip = True
        for i in range(n - 1, -1, -1):
            if a_xor_b[i] == '1' and a_and_b[i] == '0':
                a += cur
            elif a_xor_b[i] == '0' and a_and_b[i] == '1':
                a += cur
                b += cur
            elif a_xor_b[i] == '0' and a_and_b[i] == '0':
                pass
            else:
                ip = False
                break

            cur *= 2

        if not ip:
            print(-1)
        else:
            print(a, b)

6
-1


F -> 파이썬으로는 시간 초과 날 가능성 높음

In [ ]:
모두 검은색인 상태에서 흰색을 만들어 나가는 편이 나을거 같음

G

In [ ]:
1에 토큰 있으면 바로 성공

두 토큰이 보너스 선을 따라 이동할 수 있으면 성공

보너스 선을 아무 것도 못따라가면 실패

한 토큰이 보너스 선 따라갈 수 있을 때,
 나머지 토큰들 중 보너스를 계속 반복할 수 있는게 있으면 성공,
 나머지 토큰들 중 보너스로 한번 갈 수 있는 것의 개수를 구함

t1                                  t2
성공할 토큰과 1의 거리, 나머지 토큰들 중 보너스로 한번 갈 수 있는 것의 개수를 비교해야됨

t1 <= t2 + 1이면 성공함

In [31]:
from collections import deque

inf = float('inf')

# t = int(input())
# for tn in range(t):
n, m = map(int, input().split())
p, b = map(int, input().split())
tokens = set(map(int, input().split()))
bonuses = set(map(int, input().split()))

li = [[] for _ in range(n + 1)]
for _ in range(m):
    u, v = map(int, input().split())

    li[u].append(v)
    li[v].append(u)

if 1 in tokens:
    print('YES')
else:
    queue = deque()
    queue.append(1)

    dists = [inf] * (n + 1)
    dists[1] = 0

    token_dists = []
    can_token_num = -1
    while queue:
        value = queue.popleft()

        for v in li[value]:
            if dists[v] == inf:
                dists[v] = dists[value] + 1

                if v in tokens:
                    token_dists.append(dists[v])
                    can_token_num = v
                if v in bonuses:
                    queue.append(v)        

    if len(token_dists) >= 2:
        print('YES')
    elif not token_dists:
        print('NO')
    else:
        token_dist = token_dists[0]
        if token_dist == 1:
            print('YES')
        else:
            is_adj_bs = [False] * (n + 1)
            for value in range(1, n + 1):
                for v in li[value]:
                    if v in bonuses:
                        is_adj_bs[value] = True
                        break

            c_level_1 = 0
            is_level_2 = False
            for token in tokens:
                if token == can_token_num:
                    continue

                level = 0
                for v in li[token]:
                    if v in bonuses:
                        level = max(level, 1)
                        if is_adj_bs[v]:
                            level = 2

                if level == 2:
                    is_level_2 = True
                elif level == 1:
                    c_level_1 += 1

            if is_level_2:
                print('YES')
            elif token_dist <= c_level_1 + 1:
                print('YES')
            else:
                print('NO')

# if tn < t - 1:
#     input()

2 1
1 0
2

1 2
NO


NameError: name 'tn' is not defined

In [33]:
dists

[inf, 0, 1]